## Loading Example Wrapper Scripts and Manifest Files

In [50]:
import json

In [51]:
# Gather Wrapper Scripts

wrapper_scripts = {
    "Conos.VisualizeGeneExpression": "wrapper.R",
    "Kallisto.Indexer": "wrapper.py",
    "NearestTemplatePrediction": "wrapper.R",
    "NMFConsensus": "wrapper.R",
    "OutSplice.plotJunctions": "wrapper.R",
    "PreprocessVelocityTranscriptome": "wrapper.R",
    "Seurat.Clustering": "wrapper.R",
    "Seurat.IntegrateData": "wrapper.R",
    "spatialGE.Preprocessing": "wrapper.R",
    "spatialGE.STenrich": "wrapper.R",
    "ssGSEA.ROC": "wrapper.R",
    "STREAM.DimensionReduction": "wrapper.py",
    "TCGA.SampleSelection": "wrapper.R",
    "tximport.DESeq2": "wrapper.R"
}

for module_name, wrapper_fp in wrapper_scripts.items():
    with open(f'examples/{module_name}/{wrapper_fp}') as wrapper:
        wrapper_scripts[module_name] = wrapper.read()

In [52]:
modules = wrapper_scripts.keys()

In [53]:
manifests = {}

for module in modules:
    with open(f'examples/{module}/manifest') as manifest:
        manifests[module] = manifest.read()

In [54]:
finetuning_examples = [
    {"messages": [{"role": "system", "content": "You are a genius GenePattern developer who writes manifest files based on wrapper scripts. You"\
                                                "will be fired for any mistakes you make"},
                  {"role": "user", "content": wrapper_scripts[module]},
                  {"role": "assistant", "content": manifests[module]}]} for module in modules
]

In [55]:
with open("data/examples.jsonl", "w") as f:
    for item in finetuning_examples:
        f.write(json.dumps(item) + "\n")

## Finetuning Safeguards (OpenAI)

In [56]:
from collections import defaultdict
import tiktoken
import numpy as np

In [ ]:
data_path = "data/examples.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

In [59]:
encoding = tiktoken.encoding_for_model("gpt-4o")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [ ]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(finetuning_examples)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

## Finetuning

In [64]:
from openai import OpenAI

openai_api_key = ''
client = OpenAI(api_key=openai_api_key)

In [ ]:
# client.files.create(
#   file=open("data/examples.jsonl", "rb"),
#   purpose="fine-tune"
# )

In [ ]:
# client.fine_tuning.jobs.create(
#     training_file="file-NUXmngdhyQhpfMZgRQy4RV",
#     model="gpt-4o-mini-2024-07-18"
# )